<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine/CNNmodel_v3.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Builf train and evaluate
# Save model for offline scoring on test
# Model name: CCP_ICD10_Searchengine/CNNmodel_v2.0.ipynb
# References
#https://github.com/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments/Model-2-keras_lstmConv.ipynb
#https://github.com/njainds/NLP/blob/master/kaggle/QIQC/22nd%20Place%20Solution%20-%206%20models%20%2B%20POS%20tagging.ipynb
#https://github.com/Cheneng/DPCNN/blob/master/model/DPCNN.py


In [0]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [1]:

import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')

import os
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam, RMSprop
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import callbacks

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
best_model.hdf5       model_data.csv   model_v2.0.json	train_y.npy
embedding_matrix.npy  model_v1.0.h5    test_X.npy	word_index.npy
icd_dict.npy	      model_v1.0.json  test_y.npy
mispell_dict.npy      model_v2.0.h5    train_X.npy


In [0]:
os.chdir("/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/")
x_train = np.load("train_X.npy")
x_test = np.load("test_X.npy")
y_train = np.load("train_y.npy")
y_test = np.load("test_y.npy")
features = np.load("test_y.npy")
icd_dict = np.load("icd_dict.npy").item()
word_index = np.load("word_index.npy").item()
embeddings = np.load("embedding_matrix.npy")

In [0]:
class RocAucEvaluation(keras.callbacks.Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

          
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(x_test, y_test), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

In [0]:
#configs
max_features = embeddings.shape[0]
maxlen = x_train.shape[1]
embed_size = embeddings.shape[1]
n_class = y_train.shape[1]

In [22]:
del model
inp = Input(shape=(maxlen,))
x   = Embedding(max_features, embed_size, weights=[embeddings], trainable=True)(inp)
x   = SpatialDropout1D(rate=0.2)(x)
x1  = Conv1D(64, kernel_size=1, padding='valid', kernel_initializer='he_uniform')(x)
x2  = Conv1D(64, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x)
x3  = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x)
x1  = GlobalMaxPooling1D()(x1)
x2  = GlobalMaxPooling1D()(x2)
x3  = GlobalMaxPooling1D()(x3)
c   = concatenate([x1,x2,x3])
y   = Dense(128, activation='relu')(c)
y   = Dropout(0.5)(y)
out = Dense(n_class, activation='softmax')(y)
model=Model(inputs=inp,outputs=out)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=5e-4), metrics=["accuracy"])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 6, 200)       595400      input_9[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_9 (SpatialDro (None, 6, 200)       0           embedding_9[0][0]                
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 6, 64)        12864       spatial_dropout1d_9[0][0]        
__________________________________________________________________________________________________
conv1d_26 

In [23]:
gc.collect()
model.fit(x_train, y_train, batch_size = 16, epochs = 6, validation_data = (x_test, y_test), verbose = 1, callbacks = [ra_val, check_point, early_stop])


Train on 7834 samples, validate on 871 samples
Epoch 1/6
7834/7834 [==============================] - 6s 720us/step - loss: 3.8799 - acc: 0.1963 - val_loss: 2.5184 - val_acc: 0.4374

 ROC-AUC - epoch: 1 - score: 0.963427

Epoch 00001: val_loss did not improve from 0.71041
Epoch 2/6
7834/7834 [==============================] - 4s 558us/step - loss: 2.2952 - acc: 0.4728 - val_loss: 1.4364 - val_acc: 0.6739

 ROC-AUC - epoch: 2 - score: 0.985817

Epoch 00002: val_loss did not improve from 0.71041
Epoch 3/6
7834/7834 [==============================] - 4s 555us/step - loss: 1.5691 - acc: 0.6162 - val_loss: 1.0553 - val_acc: 0.7600

 ROC-AUC - epoch: 3 - score: 0.990148

Epoch 00003: val_loss did not improve from 0.71041
Epoch 4/6
7834/7834 [==============================] - 4s 558us/step - loss: 1.1958 - acc: 0.6959 - val_loss: 0.9009 - val_acc: 0.7853

 ROC-AUC - epoch: 4 - score: 0.992016

Epoch 00004: val_loss did not improve from 0.71041
Epoch 5/6
7834/7834 [============================

In [24]:
#itoicd = dict((v,k) for k,v in icd_dict.items())
test_pred = np.argmax(model.predict(x_test, batch_size = 16, verbose = 1), axis=1)
test_act = np.argmax(y_test, axis=1)
trn_pred = np.argmax(model.predict(x_train, batch_size = 16, verbose = 1), axis=1)
trn_act = np.argmax(y_train, axis=1)

print("Accuracy on val dataset is {}".format(sum(test_act==test_pred)/test_pred.shape[0]))
print("Accuracy on trn dataset is {}".format(sum(trn_act==trn_pred)/trn_pred.shape[0]))

7834/7834 [==============================] - 1s 116us/step
Accuracy on val dataset is 0.8117106773823192
Accuracy on trn dataset is 0.9098800102118969


In [25]:
# Saving Model
# serialize model to JSON
model_json = model.to_json()
with open("model_v3.0.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_v3.0.h5")
print("Saved model 3 to disk")
 
# later...
 

Saved model 3 to disk


In [0]:
# load json and create model
json_file = open('model_v3.0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_v3.0.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=5e-4), metrics=["accuracy"])
score = np.argmax(loaded_model.predict(DATASET, batch_size = 16, verbose = 1), axis=1)